In [1]:
import casadi as cs
from urdf_parser_py.urdf import URDF, Pose
import os # For current directory
import urdf2casadi.urdfparser
import urdf2casadi.geometry.plucker as plucker
import numpy as np

In [2]:
q = cs.SX.sym("q", 7)
q_dot = cs.SX.sym("q_dot", 7)
q_ddot = cs.SX.sym("q_ddot", 7)


asd = urdf2casadi.urdfparser.URDFparser()
asd.from_file("./examples/urdf/ur5.urdf")
root = "base_link"
tip = "tool0"

joint_list, nvar, actuated_names, upper, lower = asd.get_joint_info(root, tip)
i_X_0, joint_motion = asd.jcalc(root, tip, q, joint_list)
f = asd.get_inverse_dynamics(root, tip)

./examples/urdf/ur5.urdf
8


In [3]:
v = []
a = []
f = []
v.append(np.zeros(6))#kan man bruke numpy eller vanlig lise?
a.append([0., 0., 9.81, 0., 0., 0.]) #bruke numpy isteden? Og er 9.81 på riktig sted?
motion_space = [1., 0., 0., 0., 0., 0.]
#print(np.transpose(a[0]))
#print(a[0])



v.append(joint_motion[1]*q_dot[1])
print(v)
#vJ2 = cs.mtimes(motion_space,q_dot[0])
#print(vJ2)



#a.append(cs.mtimes(i_X_0[1], -np.transpose(a[0])) + cs.mtimes(motion_space, q_ddot[0]))
#v.append(cs.mtimes(i_X_0[2], v[1]) + v[1])

#a.append(cs.mtimes(i_X_0[2], a[1]) + cs.mtimes(motion_space,q_ddot[0]) + cs.mtimes(plucker.spatial_cross_product(v[2]),vJ))



[array([0., 0., 0., 0., 0., 0.]), SX(@1=0, [@1, @1, @1, @1, @1, @1])]


In [4]:

# urdf2casadi uses cs.MX and cs.SX, which can be hard to read as these are sparse matrices.
# This short function just makes it so that the result will be a numpy matrix
# Use for 
def cs2np(asd):
    return cs.Function("temp",[],[asd])()["o0"].toarray()
# NOTE: casadi imports numpy as np, so cs.np is numpy

In [5]:

#print asd.robot_desc
fk_dict = asd.get_forward_kinematics(root_link, end_link)
print fk_dict.keys()

NameError: name 'root_link' is not defined

In [ ]:
inertias = asd.get_spatial_inertias(root_link, end_link)
print(len(inertias))

In [ ]:
# CasADi MX symbol giving the joint symbols:
q = fk_dict["q"]
# Upper limits of the joint values
q_upper = fk_dict["upper"]
# Lower limits of the joint values
q_lower = fk_dict["lower"]
# Joint names
joint_names = fk_dict["joint_names"]
print "Number of joints:", q.size()[0]
print "Upper limits:", q_upper
print "Lower limits:", q_lower
print "Joint names:", joint_names

In [ ]:
T0 = fk_dict["T_fk"]([0., 0., 0., 0., 0., 0.])
p0 = T0[:3, 3]
R0 = T0[:3, :3]
print "Transformation matrix:\n",cs2np(T0)
print "Position:\n", "x:",p0[0]," y:", p0[1], " z:", p0[2]
print "Distance from origin:\n", cs.np.linalg.norm(p0), "m"

In [ ]:
joint_list, nvar, names, upper, lower = asd.get_joint_info(root_link, end_link)
#dir(joint_list)
#print(nvar)
#print(names)
#print(upper)
#print(lower)


In [ ]:
transformations = asd.get_spatial_transforms(root_link, end_link)
#print(transformations)
print(len(transformations))
print(len(joint_list))

In [ ]:
filename = "./examples/urdf/ur5.urdf"
root_link = "base_link"
end_link = "tool0"

robot_desc = URDF.from_xml_file(filename)

chain = robot_desc.get_chain(root_link, end_link)
dir(chain)
for item in chain:
    if item in robot_desc.joint_map:
        joint = robot_desc.joint_map[item]

In [ ]:
chain = robot_desc.get_chain(root_link, end_link)
link_list = []
for item in chain:
    if item in robot_desc.link_map:
        link = robot_desc.link_map[item]
        link_list += [link]

        spatial_inertias = []

        for link in link_list:
            I = link.inertial.inertia
            #print(I)
            #print(I["Ixx"])
            #print(I.get("ixx", None))
            #print I
            dir(I)